In [1]:
! pip install pyspark
! pip install findspark

Defaulting to user installation because normal site-packages is not writeable
     |███████████████████████████████▎| 309.4 MB 58.6 MB/s eta 0:00:011 MB/s eta 0:01:40     |████████████▏                   | 120.7 MB 103.9 MB/s eta 0:00:02��██▊     | 265.1 MB 58.6 MB/s eta 0:00:01

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 316.9 MB 6.1 kB/s 
     |████████████████████████████████| 200 kB 79.0 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425367 sha256=4c870ec6e27fb3cd530d9fb9ec2672d1c9da86a649501c5cdf2d16ee20a8c94d
  Stored in directory: /home/ubuntu/.cache/pip/wheels/a6/ce/f9/17d82c92f044018df2fe30af63ac043447720d5b2cee39b40f
Successfully built pyspark
Defaulting to user installation because normal site-packages is not writeable


In [26]:
spark.stop()

In [1]:
import findspark
findspark.init()
findspark.find()

'/usr/lib/spark'

In [2]:
import pandas as pd
from collections import defaultdict
from datetime import datetime

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext

import pyspark.sql.functions as f
from pyspark.sql.types import *

In [3]:
spark = (
    SparkSession
        .builder
        .appName("OTUS")
        .config('spark.executor.cores', '2')
        .config('spark.executor.instances', '4')
        .config("spark.executor.memory", "3g")
        .config("spark.driver.memory", "2g")
        .getOrCreate()
)

sql = SQLContext(spark)

### Функция для чтения файлов

In [6]:
from pyspark.sql.types import *

struct = StructType([
    StructField("tranaction_id", IntegerType(), nullable = True),
    StructField("tx_datetime", StringType(), nullable = True), 
    StructField("customer_id", IntegerType(), nullable = True), 
    StructField("terminal_id", IntegerType(), nullable = True),
    StructField("tx_amount", DoubleType(), nullable = True), 
    StructField("tx_time_seconds", IntegerType(), nullable = True), 
    StructField("tx_time_days", IntegerType(), nullable = True),
    StructField("tx_fraud", IntegerType(), nullable = True), 
    StructField("tx_fraud_scenario", IntegerType(), nullable = True)])

def read_df(name):
    df = spark.read\
          .option("header", "true")\
          .option("inferSchema", "false")\
          .option("delimiter", ",")\
          .schema(struct)\
          .csv("s3a://hw2-mlops-course/" + name)
    
    return df

### Подгружаем список файлов

In [5]:
import pickle

with open("name_files.pickle", "rb") as output_file:
    name_files = pickle.load(output_file)

In [91]:
df = read_df(name_files[0])

df.show(5)

+-------------+-------------------+-----------+-----------+---------+---------------+------------+--------+-----------------+
|tranaction_id|        tx_datetime|customer_id|terminal_id|tx_amount|tx_time_seconds|tx_time_days|tx_fraud|tx_fraud_scenario|
+-------------+-------------------+-----------+-----------+---------+---------------+------------+--------+-----------------+
|            0|2019-08-22 06:51:03|          0|        711|    70.91|          24663|           0|       0|                0|
|            1|2019-08-22 05:10:37|          0|          0|    90.55|          18637|           0|       0|                0|
|            2|2019-08-22 19:05:33|          0|        753|    35.38|          68733|           0|       0|                0|
|            3|2019-08-22 07:21:33|          0|          0|    80.41|          26493|           0|       0|                0|
|            4|2019-08-22 09:06:17|          1|        981|   102.83|          32777|           0|       0|           

### Проверка, что есть NaN значения

In [17]:
ans = []

for name_file in name_files:
    print(f'{name_file}, {datetime.today()}')
    df = read_df(name_file)
    stat_nan = df.select([f.count(f.when(f.isnan(c), c)).alias(c) for c in df.columns]).toPandas()
    cnt = stat_nan.loc[0].sum()
    
    if cnt > 0:
        ans.append(name_file)

2019-08-22.txt, 2024-02-18 12:17:43.400678
2019-09-21.txt, 2024-02-18 12:18:27.345758
2019-10-21.txt, 2024-02-18 12:19:02.638766
2019-11-20.txt, 2024-02-18 12:19:37.316847
2019-12-20.txt, 2024-02-18 12:20:11.683497
2020-01-19.txt, 2024-02-18 12:20:45.578968
2020-02-18.txt, 2024-02-18 12:21:20.062447
2020-03-19.txt, 2024-02-18 12:21:54.302914
2020-04-18.txt, 2024-02-18 12:22:28.632852
2020-05-18.txt, 2024-02-18 12:23:02.827300
2020-06-17.txt, 2024-02-18 12:23:37.125443
2020-07-17.txt, 2024-02-18 12:24:11.605306
2020-08-16.txt, 2024-02-18 12:24:45.566586
2020-09-15.txt, 2024-02-18 12:25:19.676606
2020-10-15.txt, 2024-02-18 12:25:53.951963
2020-11-14.txt, 2024-02-18 12:26:27.918497
2020-12-14.txt, 2024-02-18 12:27:01.680075
2021-01-13.txt, 2024-02-18 12:27:35.504038
2021-02-12.txt, 2024-02-18 12:28:11.292428
2021-03-14.txt, 2024-02-18 12:28:45.227034
2021-04-13.txt, 2024-02-18 12:29:19.100334
2021-05-13.txt, 2024-02-18 12:29:52.975507
2021-06-12.txt, 2024-02-18 12:30:26.549072
2021-07-12.

In [18]:
ans

[]

### Проверка, что есть Null значения

In [45]:
nans_info = defaultdict(list)

for name_file in name_files:
    print(f'{name_file}, {datetime.today()}')
    df = read_df(name_file)
    stat_nan = df.select([f.count(f.when(f.col(c).isNull(), c)).alias(c) for c in df.columns]).toPandas()
    cols_nan = [k for k, v in stat_less_zero.to_dict().items() if v[0] > 0]
    
    if len(cols_nan) > 0:
        nans_info[name_file[:-4]] = cols_nan

2019-08-22.txt, 2024-02-18 15:17:24.140547
2019-09-21.txt, 2024-02-18 15:17:54.068840
2019-10-21.txt, 2024-02-18 15:18:12.857484
2019-11-20.txt, 2024-02-18 15:18:31.089890
2019-12-20.txt, 2024-02-18 15:18:49.468159
2020-01-19.txt, 2024-02-18 15:19:07.301522
2020-02-18.txt, 2024-02-18 15:19:25.063522
2020-03-19.txt, 2024-02-18 15:19:43.189276
2020-04-18.txt, 2024-02-18 15:20:01.021177
2020-05-18.txt, 2024-02-18 15:20:19.214991
2020-06-17.txt, 2024-02-18 15:20:37.382732
2020-07-17.txt, 2024-02-18 15:20:57.643198
2020-08-16.txt, 2024-02-18 15:21:15.720407
2020-09-15.txt, 2024-02-18 15:21:34.248310
2020-10-15.txt, 2024-02-18 15:21:52.307880
2020-11-14.txt, 2024-02-18 15:22:10.404113
2020-12-14.txt, 2024-02-18 15:22:28.530273
2021-01-13.txt, 2024-02-18 15:22:46.498642
2021-02-12.txt, 2024-02-18 15:23:04.645437
2021-03-14.txt, 2024-02-18 15:23:22.680832
2021-04-13.txt, 2024-02-18 15:23:40.886985
2021-05-13.txt, 2024-02-18 15:23:58.931798
2021-06-12.txt, 2024-02-18 15:24:16.806083
2021-07-12.

In [46]:
nans_info

defaultdict(list,
            {'2019-08-22': ['customer_id'],
             '2019-09-21': ['customer_id'],
             '2019-10-21': ['customer_id'],
             '2019-11-20': ['customer_id'],
             '2019-12-20': ['customer_id'],
             '2020-01-19': ['customer_id'],
             '2020-02-18': ['customer_id'],
             '2020-03-19': ['customer_id'],
             '2020-04-18': ['customer_id'],
             '2020-05-18': ['customer_id'],
             '2020-06-17': ['customer_id'],
             '2020-07-17': ['customer_id'],
             '2020-08-16': ['customer_id'],
             '2020-09-15': ['customer_id'],
             '2020-10-15': ['customer_id'],
             '2020-11-14': ['customer_id'],
             '2020-12-14': ['customer_id'],
             '2021-01-13': ['customer_id'],
             '2021-02-12': ['customer_id'],
             '2021-03-14': ['customer_id'],
             '2021-04-13': ['customer_id'],
             '2021-05-13': ['customer_id'],
             '

### Проверка на отриц. значения

In [19]:
cols_to_analyze = list(df.columns)
cols_to_analyze.remove('tx_datetime')

In [31]:
less_zero = defaultdict(list)

for name_file in name_files:
    print(f'{name_file}, {datetime.today()}')
    df = read_df(name_file)
    
    stat_less_zero = df.select([f.count(f.when(f.col(c) < 0, c)).alias(c) for c in df.columns]).toPandas()
    cols_less_zero = [k for k, v in stat_less_zero.to_dict().items() if v[0] > 0]
        
    if len(cols_less_zero) > 0:
        less_zero[name_file[:-4]] = cols_less_zero
            
less_zero

2019-08-22.txt, 2024-02-18 12:52:54.659663
2019-09-21.txt, 2024-02-18 12:53:25.352691
2019-10-21.txt, 2024-02-18 12:53:43.742393
2019-11-20.txt, 2024-02-18 12:54:01.235118
2019-12-20.txt, 2024-02-18 12:54:18.736504
2020-01-19.txt, 2024-02-18 12:54:35.821857
2020-02-18.txt, 2024-02-18 12:54:53.033180
2020-03-19.txt, 2024-02-18 12:55:10.363189
2020-04-18.txt, 2024-02-18 12:55:27.593856
2020-05-18.txt, 2024-02-18 12:55:44.325641
2020-06-17.txt, 2024-02-18 12:56:01.140054
2020-07-17.txt, 2024-02-18 12:56:18.711249
2020-08-16.txt, 2024-02-18 12:56:35.708009
2020-09-15.txt, 2024-02-18 12:56:52.510954
2020-10-15.txt, 2024-02-18 12:57:09.380584
2020-11-14.txt, 2024-02-18 12:57:26.370636
2020-12-14.txt, 2024-02-18 12:57:43.084823
2021-01-13.txt, 2024-02-18 12:57:59.781050
2021-02-12.txt, 2024-02-18 12:58:16.561328
2021-03-14.txt, 2024-02-18 12:58:33.551246
2021-04-13.txt, 2024-02-18 12:58:50.279371
2021-05-13.txt, 2024-02-18 12:59:07.054226
2021-06-12.txt, 2024-02-18 12:59:23.802330
2021-07-12.

defaultdict(list,
            {'2019-08-22': ['customer_id'],
             '2019-09-21': ['customer_id'],
             '2019-11-20': ['customer_id'],
             '2020-01-19': ['customer_id'],
             '2020-03-19': ['customer_id'],
             '2020-04-18': ['customer_id'],
             '2020-06-17': ['customer_id'],
             '2020-07-17': ['customer_id'],
             '2020-08-16': ['customer_id'],
             '2020-09-15': ['customer_id'],
             '2020-10-15': ['customer_id'],
             '2020-11-14': ['customer_id'],
             '2020-12-14': ['customer_id'],
             '2021-02-12': ['customer_id'],
             '2021-03-14': ['customer_id'],
             '2021-05-13': ['customer_id'],
             '2021-07-12': ['customer_id'],
             '2021-08-11': ['customer_id'],
             '2021-09-10': ['customer_id'],
             '2021-10-10': ['customer_id'],
             '2021-11-09': ['customer_id'],
             '2022-02-07': ['customer_id'],
             '

In [32]:
less_zero

defaultdict(list,
            {'2019-08-22': ['customer_id'],
             '2019-09-21': ['customer_id'],
             '2019-11-20': ['customer_id'],
             '2020-01-19': ['customer_id'],
             '2020-03-19': ['customer_id'],
             '2020-04-18': ['customer_id'],
             '2020-06-17': ['customer_id'],
             '2020-07-17': ['customer_id'],
             '2020-08-16': ['customer_id'],
             '2020-09-15': ['customer_id'],
             '2020-10-15': ['customer_id'],
             '2020-11-14': ['customer_id'],
             '2020-12-14': ['customer_id'],
             '2021-02-12': ['customer_id'],
             '2021-03-14': ['customer_id'],
             '2021-05-13': ['customer_id'],
             '2021-07-12': ['customer_id'],
             '2021-08-11': ['customer_id'],
             '2021-09-10': ['customer_id'],
             '2021-10-10': ['customer_id'],
             '2021-11-09': ['customer_id'],
             '2022-02-07': ['customer_id'],
             '

### Проверка что дата в файле не совпадает с названием файла

In [40]:
ans_date = []


for name_file in name_files:
    print(f'{name_file}, {datetime.today()}')
    df = read_df(name_file)
    
    year = int(name_file[:4])
    month = int(name_file[5:7])
    day = int(name_file[8:10])
    
    datetime_right = f"{year}-{month}-{day} 00:00:00"
    
    df = df.fillna(datetime_right, subset=['tx_datetime'])
    
    split_col = f.split(df['tx_datetime'], ' ')
    df = df.withColumn('tx_date', split_col.getItem(0))
    
    cnt = df.filter(f.col('tx_date') != name_file[:-4]).count()
    
    if cnt > 0:
        ans_date.append(name_file)
        
#ans_date

2019-08-22.txt, 2024-02-18 14:47:03.907673
2019-09-21.txt, 2024-02-18 14:47:32.164253
2019-10-21.txt, 2024-02-18 14:47:49.549300
2019-11-20.txt, 2024-02-18 14:48:06.365271
2019-12-20.txt, 2024-02-18 14:48:22.738445
2020-01-19.txt, 2024-02-18 14:48:39.048417
2020-02-18.txt, 2024-02-18 14:48:55.538434
2020-03-19.txt, 2024-02-18 14:49:11.932762
2020-04-18.txt, 2024-02-18 14:49:28.285909
2020-05-18.txt, 2024-02-18 14:49:44.718593
2020-06-17.txt, 2024-02-18 14:50:01.070948
2020-07-17.txt, 2024-02-18 14:50:18.423479
2020-08-16.txt, 2024-02-18 14:50:34.583215
2020-09-15.txt, 2024-02-18 14:50:50.631258
2020-10-15.txt, 2024-02-18 14:51:06.735237
2020-11-14.txt, 2024-02-18 14:51:22.789943
2020-12-14.txt, 2024-02-18 14:51:38.967999
2021-01-13.txt, 2024-02-18 14:51:55.100409
2021-02-12.txt, 2024-02-18 14:52:11.401246
2021-03-14.txt, 2024-02-18 14:52:27.326111
2021-04-13.txt, 2024-02-18 14:52:43.257169
2021-05-13.txt, 2024-02-18 14:52:59.255048
2021-06-12.txt, 2024-02-18 14:53:15.028428
2021-07-12.

In [41]:
ans_date

['2019-08-22.txt',
 '2019-09-21.txt',
 '2019-10-21.txt',
 '2019-11-20.txt',
 '2019-12-20.txt',
 '2020-01-19.txt',
 '2020-02-18.txt',
 '2020-03-19.txt',
 '2020-04-18.txt',
 '2020-05-18.txt',
 '2020-06-17.txt',
 '2020-07-17.txt',
 '2020-08-16.txt',
 '2020-09-15.txt',
 '2020-10-15.txt',
 '2020-11-14.txt',
 '2020-12-14.txt',
 '2021-01-13.txt',
 '2021-02-12.txt',
 '2021-03-14.txt',
 '2021-04-13.txt',
 '2021-05-13.txt',
 '2021-06-12.txt',
 '2021-07-12.txt',
 '2021-08-11.txt',
 '2021-09-10.txt',
 '2021-10-10.txt',
 '2021-11-09.txt',
 '2021-12-09.txt',
 '2022-01-08.txt',
 '2022-02-07.txt',
 '2022-03-09.txt',
 '2022-04-08.txt',
 '2022-05-08.txt',
 '2022-06-07.txt',
 '2022-07-07.txt',
 '2022-08-06.txt',
 '2022-09-05.txt',
 '2022-10-05.txt',
 '2022-11-04.txt']

 ### Проверка на выбросы (в 100 раз больше 99% перцентиля

In [39]:
less_zero = defaultdict(list)

cols_to_analyze = ['tx_amount', 'tx_time_days', 'tx_time_seconds']

for name_file in name_files:
    print(f'{name_file}, {datetime.today()}')
    df = read_df(name_file)
    
    for col in cols_to_analyze:
        perc_99 = df.approxQuantile(col, [0.99], 1)[0]
        max_val = df.agg({col: "max"}).collect()[0][0]
        
        if max_val > perc_99 * 100:
            less_zero[name_file[:-4]].append(col)
            
less_zero

2019-08-22.txt, 2024-02-18 13:42:39.936741
2019-09-21.txt, 2024-02-18 13:44:09.930184
2019-10-21.txt, 2024-02-18 13:45:29.640358
2019-11-20.txt, 2024-02-18 13:46:47.618621
2019-12-20.txt, 2024-02-18 13:48:05.793622
2020-01-19.txt, 2024-02-18 13:49:22.221613
2020-02-18.txt, 2024-02-18 13:50:38.700439
2020-03-19.txt, 2024-02-18 13:51:53.590449
2020-04-18.txt, 2024-02-18 13:53:09.089073
2020-05-18.txt, 2024-02-18 13:54:24.824682
2020-06-17.txt, 2024-02-18 13:55:40.015975
2020-07-17.txt, 2024-02-18 13:56:55.879121
2020-08-16.txt, 2024-02-18 13:58:11.792102
2020-09-15.txt, 2024-02-18 13:59:27.218821
2020-10-15.txt, 2024-02-18 14:00:50.051073
2020-11-14.txt, 2024-02-18 14:02:07.021896
2020-12-14.txt, 2024-02-18 14:03:23.232875
2021-01-13.txt, 2024-02-18 14:04:39.895538
2021-02-12.txt, 2024-02-18 14:05:56.851635
2021-03-14.txt, 2024-02-18 14:07:17.377110
2021-04-13.txt, 2024-02-18 14:08:35.069625
2021-05-13.txt, 2024-02-18 14:09:51.192824
2021-06-12.txt, 2024-02-18 14:11:07.521688
2021-07-12.

defaultdict(list,
            {'2019-08-22': ['tx_amount', 'tx_amount', 'tx_time_seconds'],
             '2019-09-21': ['tx_amount', 'tx_amount'],
             '2019-10-21': ['tx_amount', 'tx_amount'],
             '2019-11-20': ['tx_amount', 'tx_amount', 'tx_time_seconds'],
             '2019-12-20': ['tx_amount', 'tx_amount', 'tx_time_seconds'],
             '2020-01-19': ['tx_amount', 'tx_amount'],
             '2020-02-18': ['tx_amount', 'tx_amount', 'tx_time_seconds'],
             '2020-03-19': ['tx_amount', 'tx_amount'],
             '2020-04-18': ['tx_amount', 'tx_amount', 'tx_time_seconds'],
             '2020-05-18': ['tx_amount', 'tx_amount', 'tx_time_seconds'],
             '2020-06-17': ['tx_amount', 'tx_amount'],
             '2020-07-17': ['tx_amount', 'tx_amount', 'tx_time_seconds'],
             '2020-08-16': ['tx_amount', 'tx_amount'],
             '2020-09-15': ['tx_amount', 'tx_amount', 'tx_time_seconds'],
             '2020-10-15': ['tx_amount', 'tx_amount'],
     

In [ ]:
outliers = defaultdict(list)

cols_to_analyze = ['tx_time_days']

for name_file in name_files:
    print(f'{name_file}, {datetime.today()}')
    df = read_df(name_file)
    
    for col in cols_to_analyze:
        perc_99 = df.approxQuantile(col, [0.99], 1)[0]
        max_val = df.agg({col: "max"}).collect()[0][0]
        
        if max_val > perc_99 * 100:
            outliers[name_file[:-4]].append(col)

2019-08-22.txt, 2024-02-18 17:11:46.869999
2019-09-21.txt, 2024-02-18 17:12:29.736831
2019-10-21.txt, 2024-02-18 17:12:56.001285
2019-11-20.txt, 2024-02-18 17:13:21.933824
2019-12-20.txt, 2024-02-18 17:13:49.074726
2020-01-19.txt, 2024-02-18 17:14:14.280972
2020-02-18.txt, 2024-02-18 17:14:39.130475
2020-03-19.txt, 2024-02-18 17:15:13.707134
2020-04-18.txt, 2024-02-18 17:15:39.162572
2020-05-18.txt, 2024-02-18 17:16:04.048979
2020-06-17.txt, 2024-02-18 17:16:29.339308
2020-07-17.txt, 2024-02-18 17:16:55.038019
2020-08-16.txt, 2024-02-18 17:17:19.587551
2020-09-15.txt, 2024-02-18 17:17:44.031765
2020-10-15.txt, 2024-02-18 17:18:08.667428
2020-11-14.txt, 2024-02-18 17:18:39.808889
2020-12-14.txt, 2024-02-18 17:19:04.472772
2021-01-13.txt, 2024-02-18 17:19:29.070818
2021-02-12.txt, 2024-02-18 17:19:54.966512
2021-03-14.txt, 2024-02-18 17:20:21.382558
2021-04-13.txt, 2024-02-18 17:20:45.974479
2021-05-13.txt, 2024-02-18 17:21:11.275870
2021-06-12.txt, 2024-02-18 17:21:35.818060
2021-07-12.

In [ ]:
outliers

### Проверка что tx_time_seconds != tx_time_days

In [37]:
ans_time = []

for name_file in name_files:
    print(f'{name_file}, {datetime.today()}')
    
    df = read_df(name_file)
    df = df.dropna(subset=['tx_time_seconds', 'tx_time_days'])
    
    df = df.withColumn('tx_time_days_calc', (f.col('tx_time_seconds') / 3600 / 24).cast(IntegerType()))
    cnt = df.filter(f.col('tx_time_days_calc') != f.col('tx_time_days')).count()
    
    if cnt > 0:
        ans_time.append(name_file)
        
ans_time

2019-08-22.txt, 2024-02-18 13:21:08.060656
2019-09-21.txt, 2024-02-18 13:21:21.080399
2019-10-21.txt, 2024-02-18 13:21:33.696518
2019-11-20.txt, 2024-02-18 13:21:46.222815
2019-12-20.txt, 2024-02-18 13:21:58.452886
2020-01-19.txt, 2024-02-18 13:22:10.704872
2020-02-18.txt, 2024-02-18 13:22:22.800083
2020-03-19.txt, 2024-02-18 13:22:35.216220
2020-04-18.txt, 2024-02-18 13:22:47.201370
2020-05-18.txt, 2024-02-18 13:23:00.065623
2020-06-17.txt, 2024-02-18 13:23:12.274395
2020-07-17.txt, 2024-02-18 13:23:24.959460
2020-08-16.txt, 2024-02-18 13:23:37.169994
2020-09-15.txt, 2024-02-18 13:23:49.276805
2020-10-15.txt, 2024-02-18 13:24:01.403168
2020-11-14.txt, 2024-02-18 13:24:13.465804
2020-12-14.txt, 2024-02-18 13:24:25.618990
2021-01-13.txt, 2024-02-18 13:24:37.837080
2021-02-12.txt, 2024-02-18 13:24:50.798041
2021-03-14.txt, 2024-02-18 13:25:03.004145
2021-04-13.txt, 2024-02-18 13:25:15.391792
2021-05-13.txt, 2024-02-18 13:25:27.741688
2021-06-12.txt, 2024-02-18 13:25:39.834985
2021-07-12.

['2019-08-22.txt',
 '2019-09-21.txt',
 '2019-10-21.txt',
 '2019-11-20.txt',
 '2019-12-20.txt',
 '2020-01-19.txt',
 '2020-02-18.txt',
 '2020-03-19.txt',
 '2020-04-18.txt',
 '2020-05-18.txt',
 '2020-06-17.txt',
 '2020-07-17.txt',
 '2020-08-16.txt',
 '2020-09-15.txt',
 '2020-10-15.txt',
 '2020-11-14.txt',
 '2020-12-14.txt',
 '2021-01-13.txt',
 '2021-02-12.txt',
 '2021-03-14.txt',
 '2021-04-13.txt',
 '2021-05-13.txt',
 '2021-06-12.txt',
 '2021-07-12.txt',
 '2021-08-11.txt',
 '2021-09-10.txt',
 '2021-10-10.txt',
 '2021-11-09.txt',
 '2021-12-09.txt',
 '2022-01-08.txt',
 '2022-02-07.txt',
 '2022-03-09.txt',
 '2022-04-08.txt',
 '2022-05-08.txt',
 '2022-06-07.txt',
 '2022-07-07.txt',
 '2022-08-06.txt',
 '2022-09-05.txt',
 '2022-10-05.txt',
 '2022-11-04.txt']

In [38]:
ans_time

['2019-08-22.txt',
 '2019-09-21.txt',
 '2019-10-21.txt',
 '2019-11-20.txt',
 '2019-12-20.txt',
 '2020-01-19.txt',
 '2020-02-18.txt',
 '2020-03-19.txt',
 '2020-04-18.txt',
 '2020-05-18.txt',
 '2020-06-17.txt',
 '2020-07-17.txt',
 '2020-08-16.txt',
 '2020-09-15.txt',
 '2020-10-15.txt',
 '2020-11-14.txt',
 '2020-12-14.txt',
 '2021-01-13.txt',
 '2021-02-12.txt',
 '2021-03-14.txt',
 '2021-04-13.txt',
 '2021-05-13.txt',
 '2021-06-12.txt',
 '2021-07-12.txt',
 '2021-08-11.txt',
 '2021-09-10.txt',
 '2021-10-10.txt',
 '2021-11-09.txt',
 '2021-12-09.txt',
 '2022-01-08.txt',
 '2022-02-07.txt',
 '2022-03-09.txt',
 '2022-04-08.txt',
 '2022-05-08.txt',
 '2022-06-07.txt',
 '2022-07-07.txt',
 '2022-08-06.txt',
 '2022-09-05.txt',
 '2022-10-05.txt',
 '2022-11-04.txt']

### Проверим, что tx_fraud принимает только бинарный признак

In [24]:
files_fraud_corrupted = []

for name_file in name_files:
    print(f'{name_file}, {datetime.today()}')
    
    df = read_df(name_file)
    
    cnt = df.filter(f.col('tx_fraud').isin([1, 0]) == False).count()
    
    if cnt > 0:
        files_fraud_corrupted.append(name_file)

2019-08-22.txt, 2024-02-18 17:42:51.553650
2019-09-21.txt, 2024-02-18 17:43:16.372838
2019-10-21.txt, 2024-02-18 17:43:27.747872
2019-11-20.txt, 2024-02-18 17:43:38.551622
2019-12-20.txt, 2024-02-18 17:43:49.273615
2020-01-19.txt, 2024-02-18 17:44:00.104568
2020-02-18.txt, 2024-02-18 17:44:10.676881
2020-03-19.txt, 2024-02-18 17:44:21.754004
2020-04-18.txt, 2024-02-18 17:44:32.400459
2020-05-18.txt, 2024-02-18 17:44:42.988354
2020-06-17.txt, 2024-02-18 17:44:54.103159
2020-07-17.txt, 2024-02-18 17:45:16.810693
2020-08-16.txt, 2024-02-18 17:45:27.325407
2020-09-15.txt, 2024-02-18 17:45:37.857672
2020-10-15.txt, 2024-02-18 17:45:48.648553
2020-11-14.txt, 2024-02-18 17:45:59.038436
2020-12-14.txt, 2024-02-18 17:46:09.492446
2021-01-13.txt, 2024-02-18 17:46:19.885179
2021-02-12.txt, 2024-02-18 17:46:30.345747
2021-03-14.txt, 2024-02-18 17:46:40.978739
2021-04-13.txt, 2024-02-18 17:46:51.345049
2021-05-13.txt, 2024-02-18 17:47:01.824694
2021-06-12.txt, 2024-02-18 17:47:12.220713
2021-07-12.

In [25]:
files_fraud_corrupted

[]

## Сколько значений у tx_fraud_scenario?

In [22]:
scenarios = set()

for name_file in name_files:
    print(f'{name_file}, {datetime.today()}')
    
    df = read_df(name_file)
    
    varts = df.select('tx_fraud_scenario').dropDuplicates().collect()
    scenarios = scenarios.union(set([x['tx_fraud_scenario'] for x in varts]))

2019-08-22.txt, 2024-02-18 17:32:04.690129
2019-09-21.txt, 2024-02-18 17:32:29.560858
2019-10-21.txt, 2024-02-18 17:32:42.215828
2019-11-20.txt, 2024-02-18 17:32:53.882977
2019-12-20.txt, 2024-02-18 17:33:05.650166
2020-01-19.txt, 2024-02-18 17:33:17.240366
2020-02-18.txt, 2024-02-18 17:33:28.640271
2020-03-19.txt, 2024-02-18 17:33:39.700337
2020-04-18.txt, 2024-02-18 17:33:51.093896
2020-05-18.txt, 2024-02-18 17:34:02.529397
2020-06-17.txt, 2024-02-18 17:34:13.631960
2020-07-17.txt, 2024-02-18 17:34:24.938567
2020-08-16.txt, 2024-02-18 17:34:36.125435
2020-09-15.txt, 2024-02-18 17:34:47.440426
2020-10-15.txt, 2024-02-18 17:34:58.883476
2020-11-14.txt, 2024-02-18 17:35:10.380860
2020-12-14.txt, 2024-02-18 17:35:21.722513
2021-01-13.txt, 2024-02-18 17:35:33.249685
2021-02-12.txt, 2024-02-18 17:35:44.535178
2021-03-14.txt, 2024-02-18 17:35:55.656036
2021-04-13.txt, 2024-02-18 17:36:06.610954
2021-05-13.txt, 2024-02-18 17:36:17.711317
2021-06-12.txt, 2024-02-18 17:36:34.351219
2021-07-12.

In [23]:
scenarios

{0, 1, 2, 3}